In [23]:
# Import libraries
import pandas as pd
from scipy import stats
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

## Occupations data

In [24]:
# Read files
occupations = pd.read_excel('occupations.xlsx')
occupations.head()

,#,zh,en,hu
0,28.0,乘务员,flight attendant,légiutas-kísérő
1,26.0,人力資源,HR specialist,HR-es
2,10.0,会计,accountant,könyvelő
3,NaN,保姆,domestic helper,NaN
4,46.0,保安,security guard,biztonsági őr


## Hungarian data

In [25]:
# Read Hungarian participant data from prolific_hu.csv
df_prolific_hu = pd.read_csv('prolific_hu.csv')
df_prolific_hu

# Return Participant id of rows where Status is RETURNED or REJECTED
rejects = df_prolific_hu[df_prolific_hu['Status'].isin(['RETURNED', 'REJECTED'])]['Participant id'].unique()

# Turn this into a list
rejects_list = rejects.tolist()
rejects_list


['5e57a0020c3c6a14a1624031',
 '599494e7bf8bcf0001ab6973',
 '5f5a27482be30c0718bbf1e0']

In [26]:
# Read Hungarian data from hu.xlsx
df_hu = pd.read_excel('ratings_hu.xlsx')

# Define Hungarian attention checks #############################################################
hu_attention_checks = ['pincérnő', 'titkárnő', 'tanárnő', 'takarítónő', 'ápolónő', 'házvezetőnő']

# Define the mapping for ratings
rating_map = {
    'Teljesen férfi': -3,
    'Nagyrészt férfi': -2,
    'Inkább férfi': -1,
    'Semleges/egyenlő': 0,
    'Inkább női': 1,
    'Nagyrészt női': 2,
    'Teljesen női': 3
}

# Get columns to convert (skip non-rating columns)
rating_columns = df_hu.columns[8:]  # assuming first 8 columns are not ratings

# Replace and explicitly infer objects to avoid warning
for col in rating_columns:
    df_hu[col] = df_hu[col].map(rating_map)

# Drop rows where Prolific ID is in rejects_list
df_hu = df_hu[~df_hu['Prolific ID'].isin(rejects_list)]

# Count and print participants based on unique Prolific IDs
num_participants = df_hu['Prolific ID'].nunique()
print(f'Number of participants: {num_participants}')

df_hu.head()

Number of participants: 19


,ID,Start time,Completion time,Email,Name,Prolific ID,Életkor,Nem,modell,katona,...,tanár,rendőr,pilóta,házvezetőnő,recepciós,biztonsági őr,ügyész,kozmetikus,programozó,diák
0,1,2025-07-11 12:06:52,2025-07-11 12:07:43,anonymous,NaN,5ef60257cd680928de23ccae,25-35,férfi,2,-3,...,0,-2,-1,3,0,-2,0,2,-1,0
1,2,2025-07-11 12:06:58,2025-07-11 12:13:33,anonymous,NaN,5c48be0496d59b000183e68d,45-55,férfi,2,-2,...,0,-2,-2,3,1,-2,-1,3,-2,0
2,3,2025-07-11 12:14:03,2025-07-11 12:16:52,anonymous,NaN,5a913d2cf0536100017196d8,25-35,férfi,0,-2,...,0,-1,0,3,0,-2,0,2,0,0
3,4,2025-07-11 12:12:24,2025-07-11 12:21:19,anonymous,NaN,5d3449524e8363001735fc41,35-45,férfi,2,-2,...,2,-2,-2,3,2,-2,-1,3,-2,0
4,5,2025-07-11 12:16:52,2025-07-11 12:21:35,anonymous,NaN,5d3873197860c8001a106e02,25-35,férfi,0,-2,...,0,-1,-1,3,0,-1,0,2,-2,0


### Demographics

In [27]:
# Show me the ratio of "Nem" (gender) in this survey.
gender = df_hu['Nem'].value_counts(normalize=True) * 100

# Show me the ratios of "Életkor" (age) in this survey.
age = df_hu['Életkor'].value_counts(normalize=True) * 100

# Translate
gender_translation = {
    'nő': 'Female',
    'férfi': 'Male'
}
gender_labels_en = gender.index.map(gender_translation)

# Prepare data for gender pie chart
gender_pie = go.Pie(
    labels=gender_labels_en,
    values=gender.values,
    name='Gender',
    hole=0.4,
    title='Gender'
)

# Prepare data for age pie chart
age_pie = go.Pie(
    labels=age.index,
    values=age.values,
    name='Age',
    hole=0.4,
    title='Age'
)

# Create subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Gender Distribution', 'Age Distribution'])

fig.add_trace(gender_pie, 1, 1)
fig.add_trace(age_pie, 1, 2)

fig.update_traces(textinfo='percent+label')
fig.update_layout(title_text='Gender and Age Distribution')

# Add a text that shows the total number of participants
total_participants = df_hu.shape[0]
fig.add_annotation(
    text=f'Number of Participants: {total_participants}',
    xref='paper', yref='paper',
    x=0.5, y=-0.1,
    showarrow=False,
    font=dict(size=16, color='black'),
    align='center'
)

fig.show()

# Save it as html
fig.write_html('demographics_hu.html')

# Save it as image
fig.write_image('demographics_hu.png', scale=2, width=1000, height=500)

### Stats

In [28]:
# Transpose results
df_hu = df_hu[rating_columns].transpose()

# Delete rows that are not in the zh_attention_checks list
df_hu = df_hu[~df_hu.index.isin(hu_attention_checks)]

# Show the number of rows
print(f"Number of rows in transposed DataFrame: {df_hu.shape[0]}")

# Show
df_hu.head()

# Copy the DataFrame for ratings in df_ratings and delete the columns 'hu_mean' and 'hu_std'
df_ratings = df_hu.copy()
df_ratings

Number of rows in transposed DataFrame: 44


,0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19
modell,2,2,0,2,0,2,2,1,1,0,1,2,0,0,1,2,0,0,2
katona,-3,-2,-2,-2,-2,-2,-2,-2,-2,-1,-2,-1,0,-1,-2,-2,-1,-2,-3
kórboncnok,0,-1,0,-1,-2,-1,-1,-2,-2,0,-1,-2,0,-1,0,0,0,0,-2
vezérigazgató,-1,-1,-1,-1,0,-1,-1,-2,-2,-1,-1,0,0,-2,-2,-2,0,-2,-2
menedzser,-1,0,-1,0,0,0,-1,-1,-2,0,-1,0,0,-1,0,-2,0,-1,-1
nővér,3,3,2,3,3,3,3,2,3,1,2,1,1,2,3,3,1,1,2
szakács,0,0,-1,-1,-1,-2,-1,-1,-2,-1,-2,-2,0,-1,-1,-2,0,-1,-1
felszolgáló,0,-1,0,1,0,0,0,0,1,0,0,2,0,0,0,2,0,0,1
könyvelő,1,0,0,2,0,1,-1,1,2,0,-1,0,0,1,1,0,0,0,2
professzor,-1,-2,0,-2,0,0,-1,-2,-3,-2,-1,0,0,-1,-1,0,0,0,-1


### One Sample T-test

In [29]:
#One Sample T-test
results = []

for index, row in df_ratings.iterrows():
    ratings = row.dropna().astype(float)
    t_stat, p_value = stats.ttest_1samp(ratings, popmean=0)
    mean_rating = ratings.mean()
    results.append({
        'item': index,  # item name from index
        'mean': mean_rating,
        't_stat': t_stat,
        'p_value': p_value,
        'significant': p_value < 0.05
    })

df_results = pd.DataFrame(results)

# Print significant results
print(df_results[df_results['significant']][['item', 'mean', 'p_value']])

# Filter for non-significant ratings
not_significant = df_results[~df_results['significant']]

# Get the min and max of the mean ratings where not significant
mean_min = not_significant['mean'].min()
mean_max = not_significant['mean'].max()

# Print the range of mean ratings where the rating is not significant
print(f"\nRange of mean ratings where the rating is not significant: {mean_min} to {mean_max}")


               item      mean       p_value
0            modell  1.052632  8.605733e-05
1            katona -1.789474  2.217099e-09
2        kórboncnok -0.842105  3.453470e-04
3     vezérigazgató -1.157895  3.377181e-06
4         menedzser -0.631579  7.944877e-04
5             nővér  2.210526  1.377051e-09
6           szakács -1.052632  4.050639e-06
8          könyvelő  0.473684  3.487999e-02
9        professzor -0.894737  5.828713e-04
10          építész -1.263158  5.952803e-06
11            tudós -0.421053  7.298826e-03
13        pénztáros  1.052632  1.431979e-05
14             bíró -0.263158  2.071954e-02
15           munkás -1.315789  1.572128e-06
16        vízimentő -1.105263  3.158723e-05
18          tűzoltó -2.210526  3.675225e-10
19           mérnök -1.052632  3.856090e-05
20          rendező -0.789474  1.359326e-04
21         takarító  1.157895  1.684414e-04
22            HR-es  0.947368  1.233705e-04
23        házvezető  1.789474  2.498062e-07
24  légiutas-kísérő  1.421053  7

In [30]:
# Sort by mean for better readability
df_results_sorted = df_results.sort_values(by='mean')

# Create color labels
df_results_sorted['significance'] = df_results_sorted['significant'].map({True: 'Significant', False: 'Not Significant'})

# Plot
fig = px.bar(
    df_results_sorted,
    x='item',
    y='mean',
    color='significance',
    color_discrete_map={'Significant': 'crimson', 'Not Significant': 'lightgray'},
    title='One Sample T-test of Ratings from Likert Scale',
    labels={'item': 'Item', 'mean': 'Mean Rating'},
    hover_data=['p_value']
)

fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_title='Mean Rating (Bias)',
    xaxis_title='Item',
    template='plotly_white'
)

fig.show()

# Save it as html
fig.write_html('occupations_ttest_hu.html')

# Save it as image  
fig.write_image('occupations_ttest_hu.png', scale=2, width=1000, height=500)

### Merge

In [31]:
# Turn index column into a column called 'hu'
df_hu.reset_index(inplace=True)
df_hu.rename(columns={'index': 'hu'}, inplace=True)

# Merge df_hu and occupations on the 'hu' column
df_hu = pd.merge(df_hu, occupations, on='hu', how='left')

# Reorder columns so the dataframe starts with 'hu' 'en', 'zh', and so on
df_hu = df_hu[['#', 'hu', 'en', 'zh'] + [col for col in df_hu.columns if col not in ['#', 'hu', 'en' , 'zh']]]

# Merge df_hu and df_results on the 'hu' column
df_hu = pd.merge(df_hu, df_results, left_on='hu', right_on='item', how='left')

# Sort all occupations by their average ratings
df_hu = df_hu.sort_values(by='mean', ascending=False)

# Drop the 'item' column as it is redundant now
df_hu.drop(columns=['item', 't_stat'], inplace=True)

# Rename the columns for clarity
df_hu.rename(columns={'mean': 'hu_mean',
                      't_stat': 'hu_t_stat',
                      'p_value': 'hu_p_value',
                      'significant': 'hu_significant'}, inplace=True)

# Save df_hu as an Excel file
df_hu.to_excel('occupations_hu.xlsx', index=False)

# Show the final DataFrame
df_hu.head()

,#,hu,en,zh,0,1,2,3,4,5,...,13,14,15,16,17,18,19,hu_mean,hu_p_value,hu_significant
5,6.0,nővér,nurse,护士,3,3,2,3,3,3,...,1,2,3,3,1,1,2,2.210526,1.377051e-09,True
41,48.0,kozmetikus,beautician,美容师,2,3,2,3,2,2,...,1,2,2,3,2,3,2,2.210526,5.888900e-13,True
23,27.0,házvezető,housekeeper,家政员,1,2,0,3,2,2,...,2,2,2,3,2,3,2,1.789474,2.498062e-07,True
24,28.0,légiutas-kísérő,flight attendant,乘务员,1,2,1,3,0,2,...,0,1,0,2,1,1,2,1.421053,7.381720e-07,True
21,25.0,takarító,cleaner,保洁,1,2,0,3,0,1,...,0,2,2,2,0,2,1,1.157895,1.684414e-04,True


### Plot

In [32]:
# Plot the Hungarian data

# Add a color column based on rating sign: feminine (rating > 0), masculine (rating < 0), neutral (rating == 0)
df_hu['bias'] = df_hu['hu_mean'].apply(
    lambda x: 'Feminine' if x > mean_max else ('Masculine' if x < mean_min else 'Neutral')
)

color_map = {
    'Feminine': '#e377c2',   # pinkish
    'Masculine': '#1f77b4',  # blue
    'Neutral': '#7f7f7f'     # gray
}

fig = px.bar(
    df_hu,
    x='hu',
    y='hu_mean',
    color='bias',
    color_discrete_map = color_map,
    title='Average Rating by Occupation (Gender Bias Highlighted)',
    labels={'hu': 'Chinese', 'en': 'English', 'bias': 'Bias', 'hu_mean': 'Rating',},
    hover_data=['zh', 'en', 'hu_mean']
)
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis=dict(
        range=[-3, 3],
        tickvals=[-3, -2, -1, 0, 1, 2, 3],
        title='Average Rating'
    )
)
fig.show()

# Save this as a html file
fig.write_html('occupations_hu.html')

# Save this as an image
fig.write_image('occupations_hu.png', scale=2, width=1000, height=500)

## Chinese data

In [33]:
# Read Chinese data from hu.xlsx
df_zh = pd.read_excel('ratings_zh.xlsx')

# Define Hungarian attention checks #######################################
zh_attention_checks = ['妈妈', '女画家', '女作家', '爸爸', '男演员', '男作家']

# Define the mapping for ratings
rating_map = {
    '完全由男性担任': -3,
    '大多由男性担任': -2,
    '较多由男性担任': -1,
    '男女比例大致相当': 0,
    '较多由女性担任': 1,
    '大多由女性担任': 2,
    '完全由女性担任': 3
}

# Get columns to convert (skip non-rating columns)
rating_columns = df_zh.columns[1:]  # assuming first 8 columns are not ratings

# Replace and explicitly infer objects to avoid warning
for col in rating_columns:
    df_zh[col] = df_zh[col].map(rating_map)

# Drop ID column
df_zh = df_zh.drop(columns=['ID'])
df_zh

,警察,秘书,教授,护士,高管,教师,前台,工人,公关,幼师,...,军人,董事长,爸爸,男演员,消防员,科学家,男作家,检察官,救生员,建筑师
0,-2,1,0,1,0,1,1,-1,0,2,...,-2,0,-3,-3,-2,0,-3,0,0,0
1,-2,2,0,2,0,0,2,0,0,2,...,-2,-2,-3,-3,-2,-2,-3,-1,-2,-2
2,-1,2,0,2,0,1,1,-2,0,2,...,-2,-1,-3,-3,-1,-1,-3,-1,-2,-1
3,-1,0,-1,2,-1,1,1,0,0,1,...,-2,-1,-3,-3,-2,-1,-3,0,-1,-1
4,-2,2,-1,2,-1,2,2,-3,0,2,...,-1,-2,-3,-3,-2,-1,-3,-1,-2,-2
5,-2,2,-2,2,-2,2,2,-2,2,2,...,-2,-2,-3,-3,-2,-2,-3,-2,-2,-2
6,-2,2,-1,2,-2,0,2,0,1,2,...,-1,-1,-3,-3,-3,0,-3,0,-2,-2
7,-2,1,0,2,-1,0,1,-1,0,2,...,-2,-1,-3,-3,-1,-2,-3,-2,-1,-2
8,-1,-1,0,2,0,1,1,-2,1,1,...,-1,0,-3,-3,-1,0,-3,0,-1,-1
9,-1,2,-1,2,-1,1,3,-1,1,2,...,-1,-1,-3,-3,-1,-1,-3,0,-1,-2


### Stats

In [34]:
# Transpose results
df_zh = df_zh[rating_columns].transpose()

# Delete rows that are not in the zh_attention_checks list
df_zh = df_zh[~df_zh.index.isin(zh_attention_checks)]

# Show the number of rows
print(f"Number of rows in transposed DataFrame: {df_zh.shape[0]}")

# Show
df_zh.head()

# Copy the DataFrame for ratings in df_ratings and delete the columns 'hu_mean' and 'hu_std'
df_ratings = df_zh.copy()
df_ratings

Number of rows in transposed DataFrame: 40


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
警察,-2,-2,-1,-1,-2,-2,-2,-2,-1,-1,-2,-3,-2,-3,-2,-2,-2
秘书,1,2,2,0,2,2,2,1,-1,2,2,3,1,1,1,2,1
教授,0,0,0,-1,-1,-2,-1,0,0,-1,0,-1,-2,-2,0,0,0
护士,1,2,2,2,2,2,2,2,2,2,1,3,2,2,2,2,2
高管,0,0,0,-1,-1,-2,-2,-1,0,-1,0,-2,-1,-1,0,0,-1
教师,1,0,1,1,2,2,0,0,1,1,1,3,1,3,2,0,0
前台,1,2,1,1,2,2,2,1,1,3,2,2,1,1,2,3,0
工人,-1,0,-2,0,-3,-2,0,-1,-2,-1,-3,-3,-2,-3,0,-2,-2
公关,0,0,0,0,0,2,1,0,1,1,0,2,-1,1,2,0,1
幼师,2,2,2,1,2,2,2,2,1,2,2,3,2,3,2,2,2


### One Sample T-test

In [35]:
#One Sample T-test
results = []

for index, row in df_ratings.iterrows():
    ratings = row.dropna().astype(float)
    t_stat, p_value = stats.ttest_1samp(ratings, popmean=0)
    mean_rating = ratings.mean()
    results.append({
        'item': index,  # item name from index
        'mean': mean_rating,
        't_stat': t_stat,
        'p_value': p_value,
        'significant': p_value < 0.05
    })

df_results = pd.DataFrame(results)

# Print significant results
print(df_results[df_results['significant']][['item', 'mean', 'p_value']])

# Filter for non-significant ratings
not_significant = df_results[~df_results['significant']]

# Get the min and max of the mean ratings where not significant
mean_min = not_significant['mean'].min()
mean_max = not_significant['mean'].max()

# Print the range of mean ratings where the rating is not significant
print(f"\nRange of mean ratings where the rating is not significant: {mean_min} to {mean_max}")


   item      mean       p_value
0    警察 -1.882353  6.926729e-10
1    秘书  1.411765  1.280319e-05
2    教授 -0.647059  3.701534e-03
3    护士  1.941176  2.762582e-12
4    高管 -0.764706  6.924293e-04
5    教师  1.117647  2.710431e-04
6    前台  1.588235  3.801807e-07
7    工人 -1.588235  2.510694e-05
8    公关  0.588235  1.319208e-02
9    幼师  2.000000  1.827768e-11
10   模特  1.117647  1.395229e-04
11   护工  1.058824  1.005465e-03
12   保姆  2.117647  1.092694e-09
13   会计  0.705882  1.336629e-02
14  工程师 -1.352941  2.524732e-06
15   保洁  1.294118  2.958650e-04
17  导购员  1.294118  2.697127e-05
18  美容师  1.882353  6.926729e-10
19  服务员  0.882353  6.206554e-04
20  乘务员  1.117647  1.395229e-04
21  理发师 -1.058824  2.510694e-05
22  空服员  1.000000  7.969979e-04
23  售票员  1.000000  7.969979e-04
24   厨师 -1.529412  1.601761e-07
25  营养师  0.588235  1.319208e-02
26  家政员  1.235294  4.096206e-03
27  收银员  1.529412  6.696565e-07
28   医生 -0.705882  1.803579e-02
29   法医 -0.823529  4.096206e-03
30  程序员 -1.470588  8.348424e-06
31   保安 

In [36]:
# Sort by mean for better readability
df_results_sorted = df_results.sort_values(by='mean')

# Create color labels
df_results_sorted['significance'] = df_results_sorted['significant'].map({True: 'Significant', False: 'Not Significant'})

# Plot
fig = px.bar(
    df_results_sorted,
    x='item',
    y='mean',
    color='significance',
    color_discrete_map={'Significant': 'crimson', 'Not Significant': 'lightgray'},
    title='One Sample T-test of Ratings from Likert Scale',
    labels={'item': 'Item', 'mean': 'Mean Rating'},
    hover_data=['p_value']
)

fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_title='Mean Rating (Bias)',
    xaxis_title='Item',
    template='plotly_white'
)

fig.show()

# Save it as html
fig.write_html('occupations_ttest_zh.html')

# Save it as image
fig.write_image('occupations_ttest_zh.png', scale=2, width=1000, height=500)

### Merge

In [37]:
# Turn index column into a column called 'zh'
df_zh.reset_index(inplace=True)
df_zh.rename(columns={'index': 'zh'}, inplace=True)

# Merge df_zh and occupations on the 'zh' column
df_zh = pd.merge(df_zh, occupations, on='zh', how='left')

# Reorder columns so the dataframe starts with 'hu' 'en', 'zh', and so on
df_zh = df_zh[['#', 'hu', 'en', 'zh'] + [col for col in df_zh.columns if col not in ['#', 'hu', 'en' , 'zh']]]

# Merge df_zh and df_results on the 'hu' column
df_zh = pd.merge(df_zh, df_results, left_on='zh', right_on='item', how='left')

# Sort all occupations by their average ratings
df_zh = df_zh.sort_values(by='mean', ascending=False)

# Drop the 'item' column as it is redundant now
df_zh.drop(columns=['item', 't_stat'], inplace=True)

# Rename the columns for clarity
df_zh.rename(columns={'mean': 'zh_mean',
                      't_stat': 'zh_t_stat',
                      'p_value': 'zh_p_value',
                      'significant': 'zh_significant'}, inplace=True)

# Save df_zh as an Excel file
df_zh.to_excel('occupations_zh.xlsx', index=False)

# Show the final DataFrame
df_zh.head()

,#,hu,en,zh,0,1,2,3,4,5,...,10,11,12,13,14,15,16,zh_mean,zh_p_value,zh_significant
12,NaN,NaN,domestic helper,保姆,2,2,1,2,2,3,...,2,3,3,2,2,3,1,2.117647,1.092694e-09,True
9,NaN,NaN,kindergarten teacher,幼师,2,2,2,1,2,2,...,2,3,2,3,2,2,2,2.000000,1.827768e-11,True
3,6.0,nővér,nurse,护士,1,2,2,2,2,2,...,1,3,2,2,2,2,2,1.941176,2.762582e-12,True
18,48.0,kozmetikus,beautician,美容师,2,2,2,1,2,2,...,2,2,3,3,2,2,1,1.882353,6.926729e-10,True
6,45.0,recepciós,receptionist,前台,1,2,1,1,2,2,...,2,2,1,1,2,3,0,1.588235,3.801807e-07,True


### Plot

In [38]:
# Plot the Hungarian data

# Add a color column based on rating sign: feminine (rating > 0), masculine (rating < 0), neutral (rating == 0)
df_zh['bias'] = df_zh['zh_mean'].apply(
    lambda x: 'Feminine' if x > mean_max else ('Masculine' if x < mean_min else 'Neutral')
)

color_map = {
    'Feminine': '#e377c2',   # pinkish
    'Masculine': '#1f77b4',  # blue
    'Neutral': '#7f7f7f'     # gray
}

fig = px.bar(
    df_zh,
    x='zh',
    y='zh_mean',
    color='bias',
    color_discrete_map = color_map,
    title='Average Rating by Occupation (Gender Bias Highlighted)',
    labels={'zh': 'Chinese', 'en': 'English', 'bias': 'Bias', 'zh_mean': 'Rating',},
    hover_data=['zh', 'en', 'zh_mean']
)
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis=dict(
        range=[-3, 3],
        tickvals=[-3, -2, -1, 0, 1, 2, 3],
        title='Average Rating'
    )
)
fig.show()

# Save this as a html file
fig.write_html('occupations_zh.html')

# Save this as an image
fig.write_image('occupations_zh.png', scale=2, width=1000, height=500)

## Compare

In [39]:
# Get the set of English occupation names from both dataframes
en_hu = set(df_hu['en'].dropna()) if 'en' in df_hu.columns else set()
en_zh = set(df_zh['en'].dropna()) if 'en' in df_zh.columns else set()

# Items only in Hungarian data
only_in_hu = en_hu - en_zh
# Items only in Chinese data
only_in_zh = en_zh - en_hu
# Items in both
in_both = en_hu & en_zh

print(f"Items only in Hungarian data ({len(only_in_hu)}): {sorted(only_in_hu)}\n")
print(f"Items only in Chinese data ({len(only_in_zh)}): {sorted(only_in_zh)}\n")
print(f"Items in both ({len(in_both)}): {sorted(in_both)}")

Items only in Hungarian data (7): ['(male) nurse', 'HR specialist', 'farmer', 'gardener', 'pilot', 'student', 'waiter*']

Items only in Chinese data (3): ['domestic helper', 'flight attendant*', 'kindergarten teacher']

Items in both (37): ['CEO', 'PR specialist', 'accountant', 'architect', 'beautician', 'caretaker', 'cashier', 'chef', 'cleaner', 'dietitian', 'director', 'doctor', 'engineer', 'firefighter', 'flight attendant', 'hairdresser', 'housekeeper', 'judge', 'lifeguard', 'manager', 'model', 'nurse', 'pathologist', 'police officer', 'professor', 'programmer', 'prosecutor', 'receptionist', 'scientist', 'secretary', 'security guard', 'shop assistant', 'soldier', 'teacher', 'ticketseller', 'waiter', 'worker']


In [ ]:
# Create a unified DataFrame with all unique occupation words (by English name)
all_en = sorted(en_hu | en_zh)

# Merge Hungarian and Chinese data on 'en' (English occupation name)
df_hu_ratings = df_hu[['en', 'hu', 'hu_mean', 'hu_significant']].copy()
df_zh_ratings = df_zh[['en', 'zh', 'zh_mean', 'zh_significant']].copy()

# Outer merge to include all occupations from both datasets
df_unified = pd.DataFrame({'en': all_en})
df_unified = df_unified.merge(df_hu_ratings, on='en', how='left')
df_unified = df_unified.merge(df_zh_ratings, on='en', how='left')

# Mark in a new column if both are significant
df_unified['both_significant'] = df_unified.apply(
    lambda row: row['hu_significant'] and row['zh_significant'] if pd.notna(row['hu_significant']) and pd.notna(row['zh_significant']) else False, axis=1)

# Drop hu_significant and zh_significant columns
df_unified.drop(columns=['hu_significant', 'zh_significant'], inplace=True)

# Add new column that shows the difference between hu_mean and zh_mean
df_unified['mean_difference'] = df_unified.apply(
    lambda row: row['hu_mean'] - row['zh_mean'] if pd.notna(row['hu_mean']) and pd.notna(row['zh_mean']) else None, axis=1)

# Sort by this
df_unified = df_unified.sort_values(by='mean_difference', ascending=False)

# Count and print the number of unique occupations in the unified DataFrame
num_unique_occupations = df_unified['en'].nunique()
print(f'Number of unique occupations in unified DataFrame: {num_unique_occupations}')

# Export this as an Excel file
df_unified.to_excel('occupations_unified.xlsx', index=False)

# Show the unified DataFrame
df_unified.head()

Number of unique occupations in unified DataFrame: 47


,en,hu,hu_mean,zh,zh_mean,both_significant,mean_difference
0,(male) nurse,ápoló,0.421053,NaN,NaN,False,NaN
1,CEO,vezérigazgató,-1.157895,董事长,-1.235294,True,0.077399
2,HR specialist,HR-es,0.947368,NaN,NaN,False,NaN
3,PR specialist,PR munkatárs,0.263158,公关,0.588235,False,-0.325077
4,accountant,könyvelő,0.473684,会计,0.705882,True,-0.232198


## Compare

In [41]:
# Prepare comparison DataFrame for occupations present in both datasets
compare_df = df_hu[df_hu['en'].isin(in_both)][['en', 'hu_mean', 'hu']].merge(
    df_zh[df_zh['en'].isin(in_both)][['en', 'zh_mean', 'zh']], on='en', suffixes=('_hu', '_zh')
)

# Sort by the average of the two means for better visualization
compare_df['mean_avg'] = (compare_df['hu_mean'] + compare_df['zh_mean']) / 2
compare_df = compare_df.sort_values('mean_avg', ascending=False)

# Create bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=compare_df['en'],
    y=compare_df['hu_mean'],
    name='Hungarian',
    marker_color="#1f7211",
    hovertemplate='Hungarian: %{customdata[0]}<br>Mean: %{y:.2f}',
    customdata=compare_df[['hu']]
))

fig.add_trace(go.Bar(
    x=compare_df['en'],
    y=compare_df['zh_mean'],
    name='Chinese',
    marker_color="#e81818",
    hovertemplate='Chinese: %{customdata[0]}<br>Mean: %{y:.2f}',
    customdata=compare_df[['zh']]
))

fig.update_layout(
    barmode='group',
    title='Comparison of Gender Bias Ratings by Occupation (Hungarian vs Chinese)',
    xaxis_title='Occupation (English)',
    yaxis_title='Mean Rating',
    xaxis_tickangle=-45,
    template='plotly_white'
)

fig.show()

# Save as html
fig.write_html('occupations_comparison.html')

# Save as image
fig.write_image('occupations_comparison.png', scale=2, width=1000, height=500)